In [ ]:
####! In preparation for Stubbs test to compare WFS with centroid methods to collimate the telescope 

## This notebook was just edited by PI and not created by.

In [ ]:
import logging
import sys

BLACK, RED, GREEN, YELLOW, BLUE, MAGENTA, CYAN, WHITE = range(8)

RESET_SEQ = '\033[0m'
COLOR_SEQ = '\033[1;%dm'
BOLD_SEQ = '\033[1m'

COLORS = {
    'WARNING': YELLOW,
    'INFO': WHITE,
    'DEBUG': BLUE,
    'CRITICAL': RED,
    'ERROR': RED
}

class MyLogFormatter(logging.Formatter):
    """ Custom logging formatter that mimics jupyter's server logging. """
    def __init__(self, fmt=" [%(levelname).1s %(asctime)s %(name)s] %(message)s", 
                 datefmt="%Y-%m-%d %H:%M:%S", use_colours=True):

        logging.Formatter.__init__(self, fmt, datefmt=datefmt)
        self.use_colours = use_colours

    @staticmethod
    def color_format(message, levelname, left_char="[", right_char="]"):

        colour = COLOR_SEQ % (30 + COLORS[levelname])

        message = message.replace(left_char, "{:s} {:s}".format(colour, left_char))
        message = message.replace(right_char, "{:s} {:s}".format(right_char, RESET_SEQ))

        return message

    def format(self, record):

        # Call the original formatter class to do the grunt work
        result = logging.Formatter.format(self, record)

        if self.use_colours:
            result = self.color_format(result, record.levelname)

        return result

custom_formatter = MyLogFormatter()
stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setFormatter(custom_formatter)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.WARNING  # We leave this quiet for now
logger.propagate = False

In [ ]:
import os
import sys
import copy
import asyncio
import logging

from lsst.ts import salobj
from lsst.ts import idl
from lsst.ts.observatory.control.auxtel import ATCS, LATISS
from lsst.ts.observatory.control.utils.enums import RotType
%config IPCompleter.use_jedi = False

In [ ]:
Initiate and configure script

In [ ]:
from lsst.ts.externalscripts.auxtel.build_pointing_model import BuildPointingModel
from lsst.ts.idl.enums.Script import ScriptState
import yaml

In [ ]:
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
script = LatissCWFSAlign(index=3, remotes=True)  # this essentially calls the init method

await script.start_task  # make sure all remotes etc are running

# Elevation 70, slew to a 7-magnitude or brighter star, and track.

In [ ]:
target_name = await script.atcs.find_target(az,70,mag_limit=7.0)
print(target_name)

In [ ]:
await script.atcs.slew_object(name = target_name, rot_type=RotType.PhysicalSky)

In [ ]:
# Take short exposure to check star is centered in the detector
science_image = await script.script.latiss.take_object(exptime=2., n=1, filter='FELH0600',grating='empty_1')
print ('science_image = ' + str(science_image))

In [ ]:
# If not, apply correction pointing offsets to xy to center image in the detector and take another image. 
# await script.atcs.offset_xy(y=140, x=0)
science_image = await script.latiss.take_object(exptime=2., n=1, filter='FELH0600',grating='empty_1')
print ('science_image = ' + str(science_image)) 

## Run CWFS

In [ ]:
# Run acquisition script
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10
results = await script.arun()

## Now slew to a booming bright star

In [ ]:
await script.atcs.slew_target('HR 807') # or HR 1169 in an hour or so

## Centroid method: Going out of focus (z axis position range = +/- 7 mm) - To get the largest donut possible

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(z=7)

### Take an image and make sure few thousand counts per pixel in the illuminated annulus are present. Find the optimal exposure time. 

In [ ]:
await script.latiss.take_engtest(exptime=10., n=1, filter='FELH0600',grating='empty_1')

### x-y offset grid, take 3 images. Change below exp time. 

In [ ]:
hexapod_offset_scale = [
    [52.459, 0.0, 0.0],
    [0.0, 50.468, 0.0],
    [0.0, 0.0, 0.0],
]

In [ ]:
# offset manually in x and y for values of -2.5, -1, 1, 2.5
hexapod_offset = [1,0,0] # x,y,z

In [ ]:
tel_offset = np.matmul(hexapod_offset, hexapod_offset_scale) # gives el and negative az!
az_offset=-tel_offset[1]
el_offset=tel_offset[0]
print(f'az offset = {az_offset} [arcsec]')
print(f'el offset = {el_offset} [arcsec]')

In [ ]:
# offset the hexapod
await script.atcs.rem.ataos.cmd_offset.set_start(x=hexapod_offset[0], y=hexapod_offset[1], z=hexapod_offset[2])

In [ ]:
# offset the telescope
await script.atcs.offset_azel(
    az=-tel_offset[1],
    el=tel_offset[0],
    relative=True,
    persistent=True,
)

In [ ]:
# check offset is correct
await script.latiss.take_engtest(exptime=2., n=1, filter='FELH0600',grating='empty_1')

In [ ]:
# take high SNR image
await script.latiss.take_engtest(exptime=2., n=1, filter='FELH0600',grating='empty_1')

In [ ]:
# declare the relative offsets of the hexapod

relative_offsets = [-1, -1.5, 2.5, 1, 1.5, -2.5]

for off in relative_offsets:
    hexapod_offset = [off,0,0] # x,y,z
    tel_offset = np.matmul(hexapod_offset, hexapod_offset_scale) # gives el and negative az!
    az_offset=-tel_offset[1]
    el_offset=tel_offset[0]
    print(f'az offset = {az_offset} [arcsec]')
    print(f'el offset = {el_offset} [arcsec]')
    # offset the hexapod
    await script.atcs.rem.ataos.cmd_offset.set_start(x=hexapod_offset[0], y=hexapod_offset[1], z=0)
    # offset the telescope
    await script.atcs.offset_azel(
        az=-tel_offset[1],
        el=tel_offset[0],
        relative=True,
        persistent=True,
    )
    # take high SNR image
    await script.latiss.take_engtest(exptime=30, n=1, filter='FELH0600',grating='empty_1')

    # offset the hexapod
    await script.atcs.rem.ataos.cmd_offset.set_start(x=-hexapod_offset[0], y=-hexapod_offset[1], z=0)
    # offset the telescope
    await script.atcs.offset_azel(
        az=-tel_offset[1],
        el=tel_offset[0],
        relative=True,
        persistent=True,
    )